# Tool Output Utilization Evaluator

### Getting Started
This sample demonstrates how to use tool output utilization evaluator on agent data. The supported input formats include:
- simple data such as strings and `dict` describing agent responses;
- user-agent conversations in the form of list of agent messages. 

Before you begin:
```bash
pip install azure-ai-evaluation
```
Set these environment variables with your own values:
1) **MODEL_DEPLOYMENT_NAME** - The deployment name of the model for this AI-assisted evaluator, as found under the "Name" column in the "Models + endpoints" tab in your Azure AI Foundry project.
2) **AZURE_OPENAI_ENDPOINT** - Azure Open AI Endpoint to be used for evaluation.
3) **AZURE_OPENAI_API_KEY** - Azure Open AI Key to be used for evaluation.
4) **AZURE_OPENAI_API_VERSION** - Azure Open AI Api version to be used for evaluation.


The Tool Output Utilization Evaluator assesses how effectively an AI agent utilizes the outputs from tools and whether it accurately incorporates this information into its responses.

Scoring is based on two levels:
1. Pass - The agent effectively utilizes tool outputs and accurately incorporates the information into its response.
2. Fail - The agent fails to properly utilize tool outputs or incorrectly incorporates the information into its response.

The evaluation includes the score, a brief explanation, and a final pass/fail result.

This evaluation focuses on measuring whether the agent properly uses the information returned by tools to provide accurate and complete responses to users.

Tool Output Utilization requires following input:
- Query - This can be a single query or a list of messages(conversation history with agent). The original task request from the user.
- Response - Response from Agent (or any GenAI App). This can be a single text response or a list of messages generated as part of Agent Response.
- Tool Definitions - Tool(s) definition used by Agent to answer the query. Required to understand the context and expected outputs of the tools.


### Initialize Tool Output Utilization Evaluator


In [ ]:
import os
from azure.ai.evaluation._evaluators._tool_output_utilization import _ToolOutputUtilizationEvaluator
from azure.ai.evaluation import AzureOpenAIModelConfiguration
from pprint import pprint

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["MODEL_DEPLOYMENT_NAME"],
)


tool_output_utilization = _ToolOutputUtilizationEvaluator(model_config=model_config)

### Samples

#### Evaluating Good Tool Output Utilization

In [ ]:
query = "How is the weather in Seattle?"
response = [
    {
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_weather_123",
                "name": "fetch_weather",
                "arguments": {"location": "Seattle"},
            }
        ],
    },
    {
        "tool_call_id": "call_weather_123",
        "role": "tool",
        "content": [{"type": "tool_result", "tool_result": {"temperature": "15°C", "condition": "partly cloudy", "humidity": "68%", "wind": "8 mph NW"}}],
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "The current weather in Seattle is partly cloudy with a temperature of 15°C. The humidity is at 68% and there's a light wind from the northwest at 8 mph.",
            }
        ],
    }
]

tool_definitions = [
    {
        "name": "fetch_weather",
        "description": "Fetches the weather information for the specified location.",
        "parameters": {
            "type": "object",
            "properties": {"location": {"type": "string", "description": "The location to fetch weather for."}},
        },
    }
]

result = tool_output_utilization(query=query, response=response, tool_definitions=tool_definitions)
pprint(result)

#### Complex Scenario with Multiple Tool Outputs

In [ ]:
query = "Can you send me an email with weather information for Seattle?"
response = [
    {
        "createdAt": "2025-03-26T17:27:35Z",
        "run_id": "run_zblZyGCNyx6aOYTadmaqM4QN",
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_CUdbkBfvVBla2YP3p24uhElJ",
                "name": "fetch_weather",
                "arguments": {"location": "Seattle"},
            }
        ],
    },
    {
        "createdAt": "2025-03-26T17:27:37Z",
        "run_id": "run_zblZyGCNyx6aOYTadmaqM4QN",
        "tool_call_id": "call_CUdbkBfvVBla2YP3p24uhElJ",
        "role": "tool",
        "content": [{"type": "tool_result", "tool_result": {"temperature": "14°C", "condition": "rainy", "precipitation": "light rain", "wind": "12 mph SW"}}],
    },
    {
        "createdAt": "2025-03-26T17:27:38Z",
        "run_id": "run_zblZyGCNyx6aOYTadmaqM4QN",
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_iq9RuPxqzykebvACgX8pqRW2",
                "name": "send_email",
                "arguments": {
                    "recipient": "your_email@example.com",
                    "subject": "Weather Information for Seattle",
                    "body": "Current weather in Seattle: 14°C and rainy with light rain. Wind is from the southwest at 12 mph. Stay dry!",
                },
            }
        ],
    },
    {
        "createdAt": "2025-03-26T17:27:41Z",
        "run_id": "run_zblZyGCNyx6aOYTadmaqM4QN",
        "tool_call_id": "call_iq9RuPxqzykebvACgX8pqRW2",
        "role": "tool",
        "content": [
            {"type": "tool_result", "tool_result": {"message": "Email successfully sent to your_email@example.com.", "status": "delivered"}}
        ],
    },
    {
        "createdAt": "2025-03-26T17:27:42Z",
        "run_id": "run_zblZyGCNyx6aOYTadmaqM4QN",
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "I have successfully sent you an email with the weather information for Seattle. The email includes the current conditions: 14°C and rainy with light rain, plus wind details. The email was delivered successfully to your inbox.",
            }
        ],
    },
]

tool_definitions = [
    {
        "name": "fetch_weather",
        "description": "Fetches the weather information for the specified location.",
        "parameters": {
            "type": "object",
            "properties": {"location": {"type": "string", "description": "The location to fetch weather for."}},
        },
    },
    {
        "name": "send_email",
        "description": "Sends an email with the specified subject and body to the recipient.",
        "parameters": {
            "type": "object",
            "properties": {
                "recipient": {"type": "string", "description": "Email address of the recipient."},
                "subject": {"type": "string", "description": "Subject of the email."},
                "body": {"type": "string", "description": "Body content of the email."},
            },
        },
    },
]

result = tool_output_utilization(query=query, response=response, tool_definitions=tool_definitions)
pprint(result)

#### Query as Conversation History

In [ ]:
# Query as conversation history instead of a single string
query_as_conversation = [
    {
        "role": "system",
        "content": "You are a helpful assistant that can fetch weather information and send emails."
    },
    {
        "role": "user", 
        "content": "Hi, I need to know the weather in Seattle for my trip planning."
    },
    {
        "role": "user",
        "content": "Can you also email me a summary so I can reference it later when packing?"
    }
]

response = [
    {
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_weather_123",
                "name": "fetch_weather",
                "arguments": {"location": "Seattle"},
            }
        ],
    },
    {
        "tool_call_id": "call_weather_123",
        "role": "tool",
        "content": [{"type": "tool_result", "tool_result": {"temperature": "15°C", "condition": "partly cloudy", "humidity": "68%", "wind": "8 mph NW", "forecast": "no rain expected"}}],
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_email_456",
                "name": "send_email",
                "arguments": {
                    "recipient": "user@example.com",
                    "subject": "Seattle Weather Summary for Trip Planning",
                    "body": "Weather summary for Seattle: Temperature is 15°C, partly cloudy conditions, humidity at 68%, with northwest winds at 8 mph. No rain is expected, so light layers should be sufficient for your trip."
                },
            }
        ],
    },
    {
        "tool_call_id": "call_email_456",
        "role": "tool",
        "content": [{"type": "tool_result", "tool_result": {"message": "Email sent successfully", "delivery_status": "delivered"}}],
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "Perfect! I've checked the weather in Seattle for your trip planning. It's currently 15°C and partly cloudy with light northwest winds and no rain expected. I've also sent you an email summary with all these details including packing recommendations based on the weather conditions. The email was delivered successfully, so you can reference it when preparing for your trip.",
            }
        ],
    }
]

tool_definitions = [
    {
        "name": "fetch_weather",
        "description": "Fetches the weather information for the specified location.",
        "parameters": {
            "type": "object",
            "properties": {"location": {"type": "string", "description": "The location to fetch weather for."}},
        },
    },
    {
        "name": "send_email",
        "description": "Sends an email with the specified subject and body to the recipient.",
        "parameters": {
            "type": "object",
            "properties": {
                "recipient": {"type": "string", "description": "Email address of the recipient."},
                "subject": {"type": "string", "description": "Subject of the email."},
                "body": {"type": "string", "description": "Body content of the email."},
            },
        },
    },
]

result = tool_output_utilization(query=query_as_conversation, response=response, tool_definitions=tool_definitions)
pprint(result)

#### Example of Poor Tool Output Utilization

In [ ]:
query = "How is the weather in Seattle?"
# Poor utilization - agent doesn't use the detailed tool output effectively
poor_response = [
    {
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_weather_123",
                "name": "fetch_weather",
                "arguments": {"location": "Seattle"},
            }
        ],
    },
    {
        "tool_call_id": "call_weather_123",
        "role": "tool",
        "content": [{"type": "tool_result", "tool_result": {"temperature": "15°C", "condition": "partly cloudy", "humidity": "68%", "wind": "8 mph NW", "UV_index": "3", "visibility": "10 km"}}],
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "The weather in Seattle is okay. It's not too bad outside.",  # Poor utilization of detailed weather data
            }
        ],
    }
]

tool_definitions = [
    {
        "name": "fetch_weather",
        "description": "Fetches the weather information for the specified location.",
        "parameters": {
            "type": "object",
            "properties": {"location": {"type": "string", "description": "The location to fetch weather for."}},
        },
    }
]

# This should score poorly due to inadequate use of tool output
result = tool_output_utilization(query=query, response=poor_response, tool_definitions=tool_definitions)
pprint(result)

#### Response as String (str)

In [ ]:
query = "What's the weather like in Seattle?"
# Response as a simple string (not a list of messages)
response_str = "The weather in Seattle is currently 15°C and partly cloudy with light winds from the northwest."

tool_definitions = [
    {
        "name": "fetch_weather",
        "description": "Fetches the weather information for the specified location.",
        "parameters": {
            "type": "object",
            "properties": {"location": {"type": "string", "description": "The location to fetch weather for."}},
        },
    }
]

result = tool_output_utilization(query=query, response=response_str, tool_definitions=tool_definitions)
pprint(result)

#### Tool Definitions as Single Dict

In [ ]:
query = "Check the weather in Seattle"
response = [
    {
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_weather_456",
                "name": "fetch_weather",
                "arguments": {"location": "Seattle"},
            }
        ],
    },
    {
        "tool_call_id": "call_weather_456",
        "role": "tool",
        "content": [{"type": "tool_result", "tool_result": {"temperature": "14°C", "condition": "rainy"}}],
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "The current weather in Seattle is rainy with a temperature of 14°C.",
            }
        ],
    }
]

# Tool definition as a single dict (not a list)
tool_definition_dict = {
    "name": "fetch_weather",
    "description": "Fetches the weather information for the specified location.",
    "parameters": {
        "type": "object",
        "properties": {"location": {"type": "string", "description": "The location to fetch weather for."}},
    },
}

result = tool_output_utilization(query=query, response=response, tool_definitions=tool_definition_dict)
pprint(result)